In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST, CIFAR10
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
import json

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")

# add reproducibility stuff
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

GPU is available


In [2]:
class SparseWrap(nn.Module):
    def __init__(self, module, intrinsic_dimension, device):
        """
        Wrapper to estimate the intrinsic dimensionality of the
        objective landscape for a specific task given a specific model
        :param module: pytorch nn.Module
        :param intrinsic_dimension: dimensionality within which we search for solution
        :param device: cuda device id
        """
        super(SparseWrap, self).__init__()

        # Hide this from inspection by get_parameters()
        self.m = [module]

        self.name_base_localname = []

        # Stores the initial value: \theta_{0}^{D}
        self.initial_value = dict()

        # Stores the randomly generated projection matrix P
        self.random_matrix = dict()

        # Parameter vector that is updated, initialised with zeros as per text: \theta^{d}
        V = nn.Parameter(
            torch.zeros((intrinsic_dimension, 1)).to(device, non_blocking=True)
        )
        self.register_parameter("V", V)
        v_size = (intrinsic_dimension,)

        # Iterates over layers in the Neural Network
        for name, param in module.named_parameters():
            # If the parameter requires gradient update
            if param.requires_grad:

                # Saves the initial values of the initialised parameters from param.data and sets them to no grad.
                # (initial values are the 'origin' of the search)
                self.initial_value[name] = v0 = (
                    param.clone()
                    .detach()
                    .requires_grad_(False)
                    .to(device, non_blocking=True)
                )

                # If v0.size() is [4, 3], then below operation makes it [4, 3, v_size]
                matrix_size = v0.size() + v_size

                # Generates random projection matrices P, sets them to no grad
                self.random_matrix[name] = (
                    torch.randn(matrix_size, requires_grad=False)
                    .to_sparse()
                    .to(device, non_blocking=True)
                    / intrinsic_dimension**0.5
                )

                # NOTE!: lines below are not clear!
                base, localname = module, name
                while "." in localname:
                    prefix, localname = localname.split(".", 1)
                    base = base.__getattr__(prefix)
                self.name_base_localname.append((name, base, localname))

        for name, base, localname in self.name_base_localname:
            delattr(base, localname)

    def forward(self, x):
        # Iterate over the layers
        for name, base, localname in self.name_base_localname:
            # print(name)
            # print(localname)

            # Product between matrix P and \theta^{d}
            # print("random_matrix shape: ", self.random_matrix[name].shape)
            # print("random_matrix: ", self.random_matrix[name])
            # print("V shape: ", self.V.shape)

            # print("initail value shape:", self.initial_value[name].shape)
            if len(self.random_matrix[name].shape) <= 2:
                ray = torch.mm(self.random_matrix[name], self.V)
            else:
                ray = torch.bmm(
                    self.random_matrix[name],
                    self.V.broadcast_to(
                        (
                            self.random_matrix[name].shape[0],
                            self.V.shape[0],
                            self.V.shape[-1],
                        )
                    ),
                )
            # print("ray shape: ",ray.shape)

            # Add the \theta_{0}^{D} to P \dot \theta^{d}
            param = torch.squeeze(ray, -1) + self.initial_value[name]
            # print("param shape: ", param.shape)

            setattr(base, localname, param)

        # Pass through the model, by getting the module from a list self.m
        module = self.m[0]
        x = module(x)
        return x

In [3]:
def fast_walsh_hadamard_torched(x, axis=0, normalize=False):
    """
    Performs fast Walsh Hadamard transform
    :param x:
    :param axis:
    :param normalize:
    :return:
    """
    orig_shape = x.size()
    assert axis >= 0 and axis < len(
        orig_shape
    ), "For a vector of shape %s, axis must be in [0, %d] but it is %d" % (
        orig_shape,
        len(orig_shape) - 1,
        axis,
    )
    h_dim = orig_shape[axis]
    h_dim_exp = int(round(np.log(h_dim) / np.log(2)))
    assert h_dim == 2**h_dim_exp, (
        "hadamard can only be computed over axis with size that is a power of two, but"
        " chosen axis %d has size %d" % (axis, h_dim)
    )

    working_shape_pre = [int(np.prod(orig_shape[:axis]))]  # prod of empty array is 1 :)
    working_shape_post = [
        int(np.prod(orig_shape[axis + 1 :]))
    ]  # prod of empty array is 1 :)
    working_shape_mid = [2] * h_dim_exp
    working_shape = working_shape_pre + working_shape_mid + working_shape_post

    ret = x.view(working_shape)

    for ii in range(h_dim_exp):
        dim = ii + 1
        arrs = torch.chunk(ret, 2, dim=dim)
        assert len(arrs) == 2
        ret = torch.cat((arrs[0] + arrs[1], arrs[0] - arrs[1]), axis=dim)

    if normalize:
        ret = ret / torch.sqrt(float(h_dim))

    ret = ret.view(orig_shape)

    return ret


def rademacher(shape, device=0):
    """Creates a random tensor of shape under the Rademacher distribution (P(x=1) == P(x=-1) == 0.5)"""
    x = torch.empty(shape, device=device, requires_grad=False).random_(
        0, 2
    )  # Creates random tensor of 0 and 1
    x[x == 0] = -1  # Turn the 0s into -1
    return x


def fastJL_vars(DD, d, device=0):
    """
    Returns parameters for fast food transform
    :param DD: desired dimension
    :return:
    """
    epsilon = 0.1
    ll = int(np.ceil(np.log2(d)))
    LL = 2**ll

    # random reflection given by the diagonal matrix D ∈ R^d×d where Dii are independent Rademacher random variables
    D = torch.diag(rademacher(LL, device=device)).to(device, non_blocking=True)
    D.requires_grad = False

    n = np.log(60000)
    k = int(np.ceil(n / epsilon**2))
    # print("k: ", k)
    # Pij ≡ bijxrij , where bij ∼ Bernoulli(q) and rij ∼ N (0, q−1) are independent random variables
    q = min(n / epsilon * LL, 1)
    B = torch.empty(
        (k, LL), dtype=torch.float32, device=device, requires_grad=False
    ).bernoulli_(q)

    R = torch.empty(
        (k, LL), dtype=torch.float32, device=device, requires_grad=False
    ).normal_(0, 1 / q)

    PP = torch.mul(B, R)
    PP.requires_grad = False
    PP.to(device, non_blocking=True)
    # print("PP: ", PP.shape)

    return [D, PP, LL]


def fastJL_torched(x, DD, param_list=None, device=0):
    """
    Fastfood transform
    :param x: array of dd dimension
    :param DD: desired dimension
    :return:
    """
    dd = x.size(0)
    # print("dd: ", dd)

    if not param_list:

        D, PP, LL = fastJL_vars(DD, dd, device=device)

    else:

        D, PP, LL = param_list

    # Padd x if needed
    dd_pad = F.pad(x, pad=(0, LL - dd), value=0, mode="constant")

    # From left to right (1/k)PH(Dx), where H is Walsh-Hadamard matrix
    mul_1 = torch.mul(D, dd_pad)
    # print("mul_1: ", mul_1.shape)

    # (1/k)P(HDx)
    mul_2 = fast_walsh_hadamard_torched(mul_1, 0, normalize=False)
    # print("mul_2: ", mul_2.shape)

    # (1/k)(PHDx)
    mul_3 = torch.mm(PP, mul_2).flatten()
    # print("mul_3: ", mul_3.shape)

    ret = 1 / dd * mul_3[:DD]

    return ret


class FastJLWrapper(nn.Module):
    def __init__(self, module, intrinsic_dimension, device):
        """
        Wrapper to estimate the intrinsic dimensionality of the
        objective landscape for a specific task given a specific model using FastJL transform
        :param module: pytorch nn.Module
        :param intrinsic_dimension: dimensionality within which we search for solution
        :param device: cuda device id
        """
        super(FastJLWrapper, self).__init__()

        # Hide this from inspection by get_parameters()
        self.m = [module]

        self.name_base_localname = []

        # Stores the initial value: \theta_{0}^{D}
        self.initial_value = dict()

        # Fastfood parameters
        self.fastJL_params = {}

        # Parameter vector that is updated
        # Initialised with zeros as per text: \theta^{d}
        V = nn.Parameter(
            torch.zeros((intrinsic_dimension), device=device)
        )  # .to(device))
        self.register_parameter("V", V)
        V.to(device, non_blocking=True)

        # Iterate over layers in the module
        for name, param in module.named_parameters():
            # If param requires grad update
            if param.requires_grad:

                # Saves the initial values of the initialised parameters from param.data and sets them to no grad.
                # (initial values are the 'origin' of the search)
                self.initial_value[name] = v0 = (
                    param.clone()
                    .detach()
                    .requires_grad_(False)
                    .to(device, non_blocking=True)
                )

                # Generate fastJL parameters
                DD = np.prod(v0.size())
                self.fastJL_params[name] = fastJL_vars(DD, V.size(0), device)

                base, localname = module, name
                while "." in localname:
                    prefix, localname = localname.split(".", 1)
                    base = base.__getattr__(prefix)
                self.name_base_localname.append((name, base, localname))

        for name, base, localname in self.name_base_localname:
            delattr(base, localname)

    def forward(self, x):
        # Iterate over layers
        for name, base, localname in self.name_base_localname:

            init_shape = self.initial_value[name].size()
            # print("init_shape: ", init_shape)
            DD = np.prod(init_shape)
            # print("DD: ", DD)

            # FastJL transform replace dense P
            ray = fastJL_torched(self.V, DD, self.fastJL_params[name]).view(init_shape)

            param = self.initial_value[name] + ray

            setattr(base, localname, param)

        # Pass through the model, by getting hte module from a list self.m
        module = self.m[0]
        x = module(x)
        return x


In [4]:
BATCH_SIZE = 128
DATASET_NAME = "MNIST"

img_transform = transforms.Compose([transforms.ToTensor()])

train_dataset = None
test_dataset = None
if DATASET_NAME == "MNIST":
    train_dataset = MNIST(
        root="./data/MNIST", download=True, train=True, transform=img_transform
    )
    test_dataset = MNIST(
        root="./data/MNIST", download=True, train=False, transform=img_transform
    )
elif DATASET_NAME == "CIFAR10":
    train_dataset = CIFAR10(
        root="./data/CIFAR10", download=True, train=True, transform=img_transform
    )
    test_dataset = CIFAR10(
        root="./data/CIFAR10", download=True, train=False, transform=img_transform
    )
else:
    raise Exception("Name of dataset not in: [MNIST, CIFAR10]")

In [5]:
if DATASET_NAME == "MNIST":
    channel_in = 1
    input_height = 28
    input_width = 28
    input_dim = input_height * input_width * channel_in
    output_dim = 10
    idx = 0  # @param {type:"slider", min:0, max:59999, step:1}
else:
    channel_in = 3
    input_height = 32
    input_width = 32
    input_dim = input_height * input_width * channel_in
    output_dim = 10
    idx = 0  # @param {type:"slider", min:0, max:49999, step:1}

px.imshow(train_dataset.data[idx])

In [6]:
# Class for a Fully Connected Network
class FullyConnectedNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(FullyConnectedNetwork, self).__init__()
        self.num_layers = num_layers
        self.fc_in = nn.Linear(input_dim, hidden_dim)
        if num_layers > 0:
            self.fcs = nn.ModuleList(
                [nn.Linear(hidden_dim, hidden_dim) for _ in range(num_layers)]
            )
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc_in(x))
        if self.num_layers > 0:
            for fc in self.fcs:
                x = F.relu(fc(x))
        x = self.fc_out(x)
        return x

In [7]:
# Class for Standard LeNet Network, reference http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf with some modification to follow the same number of parameters as the main paper for the task does
class LeNet(nn.Module):
    def __init__(self, input_dim, output_dim, dataset="mnist"):
        super(LeNet, self).__init__()
        # 6 kernels 5x5
        self.conv1 = nn.Conv2d(
            input_dim,
            6,
            5,
            padding="valid",
        )
        # max-pooling over 2x2
        self.pool1 = nn.MaxPool2d(2, stride=2)
        # 16 kernels 5x5
        self.conv2 = nn.Conv2d(6, 16, 5, padding="valid")
        # max-pooling over 2x2
        self.pool2 = nn.MaxPool2d(2, stride=2)
        # 120 kernels 4x4 to match the dimensionality of the fully connected network
        self.conv3 = nn.Conv2d(
            16,
            120,
            5 if dataset == "cifar10" else 4,
        )
        # 120 fully connected neurons, too many parameter in this case w.r.t. the paper
        # self.fc1 = nn.Linear(16 * 5 * 5, 120,)
        self.flat = nn.Flatten(start_dim=1)
        # 84 fully connected neurons
        self.fc2 = nn.Linear(120, 84)
        # 10 fully connected neurons
        self.fc3 = nn.Linear(
            84,
            output_dim,
        )

    def forward(self, x):
        # x = x.view(-1, 1, 28, 28)
        #print(x.shape)
        x = self.conv1(x)
        #print(x.shape)
        x = self.pool1(F.relu(x))
        #print(x.shape)
        x = self.pool2(F.relu(self.conv2(x)))
        #print(x.shape)
        x = F.relu(self.conv3(x))
        x = self.flat(x)
        # x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [10]:
from torch.nn.modules.utils import _pair

# from https://discuss.pytorch.org/t/locally-connected-layers/26979
class LocallyConnected2d(nn.Module):
    def __init__(
        self, in_channels, out_channels, output_size, kernel_size, stride=1, bias=True
    ):
        super(LocallyConnected2d, self).__init__()
        output_size = _pair(output_size)
        self.weight = nn.Parameter(
            nn.init.kaiming_normal_(
                torch.randn(
                    1,
                    out_channels,
                    in_channels,
                    output_size[0],
                    output_size[1],
                    kernel_size**2,
                ),
                nonlinearity="relu",
            )
        )
        if bias:
            self.bias = nn.Parameter(
                nn.init.kaiming_normal_(
                    torch.randn(1, out_channels, output_size[0], output_size[1]),
                    nonlinearity="relu",
                )
            )
        else:
            self.register_parameter("bias", None)
        self.kernel_size = _pair(kernel_size)
        self.stride = _pair(stride)

    def forward(self, x):
        _, c, h, w = x.size()
        kh, kw = self.kernel_size
        dh, dw = self.stride
        x = x.unfold(2, kh, dh).unfold(3, kw, dw)
        x = x.contiguous().view(*x.size()[:-2], -1)
        # Sum in in_channel and kernel_size dims
        out = (x.unsqueeze(1) * self.weight).sum([2, -1])
        if self.bias is not None:
            out += self.bias
        return out

' class Untied_LeNet(nn.Module):\n    def __init__(self, input_dim, output_dim):\n        super(Untied_LeNet, self).__init__()\n        # 6 kernels 5x5\n        self.conv1 = LocallyConnected2d(input_dim, 6, (24,24), 5)\n        # max-pooling over 2x2\n        self.pool1 = nn.MaxPool2d(2, stride=2)\n        # 16 kernels 5x5\n        self.conv2 = LocallyConnected2d(6, 16, (8,8), 5)\n        # max-pooling over 2x2\n        self.pool2 = nn.MaxPool2d(2, stride=2)\n        # 120 kernels 4x4 to match the dimensionality of the fully connected network\n        self.conv3 = LocallyConnected2d(16, 120, (1,1), 4)\n        # 120 fully connected neurons, too many parameter in this case w.r.t. the paper\n        #self.fc1 = nn.Linear(16 * 5 * 5, 120,)\n        self.flat = nn.Flatten(start_dim=1)\n        # 84 fully connected neurons\n        self.fc2 = nn.Linear(120, 84)\n        # 10 fully connected neurons\n        self.fc3 = nn.Linear(84, output_dim,)\n\n    def forward(self, x):\n        #x = x.v

In [11]:
# Class for general Untied LeNet Network
class Untied_LeNet(nn.Module):
    def __init__(self, input_dim, output_dim, input_height, input_width):
        super(Untied_LeNet, self).__init__()
        # 6 kernels 5x5, output size 24x24 MNIST, output size 28x28 CIFAR10
        self.kernel_size = 5
        self.pool_kernel_size = 2
        self.out_conv1 = (
            input_height - (self.kernel_size - 1),
            input_width - (self.kernel_size - 1),
        )
        self.conv1 = LocallyConnected2d(input_dim, 6, self.out_conv1, self.kernel_size)
        # max-pooling over 2x2
        self.pool1 = nn.MaxPool2d(self.pool_kernel_size, stride=2)
        # 16 kernels 5x5 output size 8x8, output size 10x10 CIFAR10
        self.out_conv2 = (
            int(self.out_conv1[0] / self.pool_kernel_size) - (self.kernel_size - 1),
            int(self.out_conv1[1] / self.pool_kernel_size) - (self.kernel_size - 1),
        )
        self.conv2 = LocallyConnected2d(6, 16, self.out_conv2, self.kernel_size)
        # max-pooling over 2x2
        self.pool2 = nn.MaxPool2d(self.pool_kernel_size, stride=2)
        # 120 kernels 4x4 to match the dimensionality of the fully connected network and obtain an output size of 1x1 for MNIST and kernels 5x5 for CIFAR10
        self.conv3 = LocallyConnected2d(
            16, 120, (1, 1), int(self.out_conv2[0] / self.pool_kernel_size)
        )
        # 120 fully connected neurons, too many parameter in this case w.r.t. the paper
        # self.fc1 = nn.Linear(16 * 5 * 5, 120,)
        self.flat = nn.Flatten(start_dim=1)
        # 84 fully connected neurons
        self.fc2 = nn.Linear(120, 84)
        # 10 fully connected neurons
        self.fc3 = nn.Linear(
            84,
            output_dim,
        )

    def forward(self, x):
        # x = x.view(-1, 1, 28, 28)
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.flat(x)
        # x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
# Class for FC-LeNet Network
class FcLeNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(FcLeNet, self).__init__()
        # 6 kernels 5x5
        self.fcconv1 = nn.Linear(input_dim, 3456)
        # max-pooling over 2x2
        self.pool1 = nn.MaxPool2d(2, stride=2)
        # 16 kernels 5x5
        self.fcconv2 = nn.Linear(864, 1024)
        # max-pooling over 2x2
        self.pool2 = nn.MaxPool2d(2, stride=2)
        # 120 kernels 4x4 to match the dimensionality of the fully connected network
        self.fcconv3 = nn.Linear(256, 120)
        # 120 fully connected neurons, too many parameter in this case w.r.t. the paper
        # self.fc1 = nn.Linear(16 * 5 * 5, 120,)
        self.flat = nn.Flatten(start_dim=1)
        # 84 fully connected neurons
        self.fc2 = nn.Linear(120, 84)
        # 10 fully connected neurons
        self.fc3 = nn.Linear(
            84,
            output_dim,
        )

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.pool1(F.relu(self.fcconv1(x)).view(-1, 6, 24, 24))
        x = self.flat(x)
        x = self.pool2(F.relu(self.fcconv2(x)).view(-1, 16, 8, 8))
        x = self.flat(x)
        x = F.relu(self.fcconv3(x))
        # x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
# Class for general FCTied-LeNet
class FCTied_LeNet(nn.Module):
    def __init__(self, input_dim, output_dim, input_height, input_width):
        super(FCTied_LeNet, self).__init__()
        # 6 kernels (2*H-1)x(2*H-1)
        self.conv1 = nn.Conv2d(
            input_dim,
            6,
            2 * input_height - 1,
            padding="same",
        )
        # max-pooling over 2x2
        self.pool1 = nn.MaxPool2d(2, stride=2)
        # 16 kernels (H-1)x(H-1)
        self.conv2 = nn.Conv2d(6, 16, input_height - 1, padding="same")
        # max-pooling over 2x2
        self.pool2 = nn.MaxPool2d(2, stride=2)
        # 120 kernels 7x7 to match the dimensionality of the fully connected network
        self.conv3 = nn.Conv2d(
            16,
            120,
            int(input_height / 4),  # TODO: check if this is correct for cifar10
        )
        # 120 fully connected neurons, too many parameter in this case w.r.t. the paper
        # self.fc1 = nn.Linear(16 * 5 * 5, 120,)
        self.flat = nn.Flatten(start_dim=1)
        # 84 fully connected neurons
        self.fc2 = nn.Linear(120, 84)
        # 10 fully connected neurons
        self.fc3 = nn.Linear(
            84,
            output_dim,
        )

    def forward(self, x):
        # x = x.view(-1, 1, 28, 28)
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.flat(x)
        # x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
hidden_dim = 200
num_layers = 1
model = FullyConnectedNetwork(input_dim, hidden_dim, output_dim, num_layers)
model.to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: ", num_params)

# save information to file
# with open('results.txt', 'a') as f:
#     f.write("\n##############################################")
#     f.write(f"\nNumber_of_parameters: {num_params}")
#     f.write(f"\nhidden_dim: {hidden_dim}")
#     f.write(f"\nnum_layers: {num_layers}")
# f.close()

Number of parameters:  199210


In [7]:
modules = [module for module in model.modules()]
# Print Model Summary
print(modules[0])

FullyConnectedNetwork(
  (fc_in): Linear(in_features=784, out_features=200, bias=True)
  (fcs): ModuleList(
    (0): Linear(in_features=200, out_features=200, bias=True)
  )
  (fc_out): Linear(in_features=200, out_features=10, bias=True)
)


In [8]:
model = LeNet(channel_in, output_dim)
model.to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: %d" % num_params)

# with open("results.txt", "a") as f:
#     f.write("\n##############################################")
#     f.write(f"\nNumber_of_parameters: {num_params}")
#     f.write("\ntype: LeNet")
# f.close()

Number of parameters: 44426


In [12]:
modules = [module for module in model.modules()]
# Print Model Summary
print(modules[0])

LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=valid)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=valid)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 120, kernel_size=(4, 4), stride=(1, 1))
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [12]:
model = Untied_LeNet(channel_in, output_dim, input_height, input_width)
model.to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: %d" % num_params)

Number of parameters: 658238


In [11]:
model = FcLeNet(input_dim, output_dim)
model.to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: %d" % num_params)

Number of parameters: 3640574


In [12]:
modules = [module for module in model.modules()]
# Print Model Summary
print(modules[0])

FcLeNet(
  (fcconv1): Linear(in_features=784, out_features=3456, bias=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fcconv2): Linear(in_features=864, out_features=1024, bias=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fcconv3): Linear(in_features=256, out_features=120, bias=True)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [11]:
model = FCTied_LeNet(channel_in, output_dim, input_height, input_width)
model.to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: %d" % num_params)

Number of parameters: 193370


In [7]:
intrinsic_dim = 150
model_intrinsic = SparseWrap(model, intrinsic_dimension=intrinsic_dim, device=device)
num_params_intrinsic = sum(
    p.numel() for p in model_intrinsic.parameters() if p.requires_grad
)
print("Number of parameters: %d" % num_params_intrinsic)

Number of parameters: 150


In [9]:
intrinsic_dim = 500
model_intrinsic = FastJLWrapper(model, intrinsic_dimension=intrinsic_dim, device=device)
num_params_intrinsic = sum(
    p.numel() for p in model_intrinsic.parameters() if p.requires_grad
)
print("Number of parameters: %d" % num_params_intrinsic)

Number of parameters: 500


In [10]:
# torch.autograd.set_detect_anomaly(True) #this line can have huge performance impact
# train the model
from logging import raiseExceptions

# training step


def train(model, train_loader, optimizer, epoch):
    model.train()
    # train_loss_averager = make_averager()  # mantain a running average of the loss

    # TRAIN
    tqdm_iterator = tqdm(
        enumerate(train_loader),
        total=len(train_loader),
        desc="",
        leave=True,
    )

    len_tr_dl_ds = len(train_loader.dataset)

    for batch_idx, (data, target) in tqdm_iterator:
        data, target = data.to(device, non_blocking=True), target.to(
            device, non_blocking=True
        )
        output = model(data)
        loss = F.cross_entropy(output, target)
        model.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

        # train_loss_averager(loss.item())
        tqdm_iterator.set_description(
            f"Train Epoch: {epoch} [ {batch_idx * len(data)}/{len_tr_dl_ds} \tLoss: {loss.item():.6f}]"
        )
        tqdm_iterator.refresh()  # to show immediately the update
    tqdm_iterator.close()

    if np.isnan(loss.item()):
        print("Loss is nan")
        raise Exception("Loss is nan")
        exit()


# testing step
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    tqdm_iterator = tqdm(
        enumerate(test_loader),
        total=len(test_loader),
        desc="",
        leave=True,
    )

    len_ts_dl_ds = len(test_loader.dataset)

    with torch.no_grad():
        for batch_idx, (data, target) in tqdm_iterator:
            data, target = data.to(device, non_blocking=True), target.to(
                device, non_blocking=True
            )
            output = model(data)
            test_loss += F.cross_entropy(output, target).item()  # sum up batch loss
            # get the index of the max probability
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).cpu().sum().item()
            tqdm_iterator.set_description(
                f"Test Epoch: {epoch} [{batch_idx * len(data)}/{len_ts_dl_ds} \tLoss: {test_loss:.6f}, Accuracy: {correct}/{len_ts_dl_ds} ({100.0 * correct / len_ts_dl_ds}%)"
            )
    tqdm_iterator.refresh()  # to show immediately the update
    test_loss /= len(test_loader.dataset)
    print(f"Validation Average loss: {test_loss:.6f}")

    tqdm_iterator.close()

    # show an histogram of the weights of the model
    """start = -1
    stop = 1
    bins = 30
    for param in model.parameters():
        if param.requires_grad:
            
            hist = torch.histc(param.data, bins = bins, min = start, max = stop)
            x = np.arange(start, stop, (stop-start)/bins)
            plt.bar(x, hist.cpu(), align='center')
            plt.ylabel('Frequency')
            plt.show() """

    return correct / len(test_loader.dataset)


if __name__ == "__main__":
    learning_rate = 0.1
    optimizer = optim.SGD(model_intrinsic.parameters(), lr=learning_rate)
    # download and load MNIST Dataset
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
    )  # persistent_workers=True)
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
    )  # persistent_workers=True)
    # train the model
    best_acc, best_epoch = 0, 0
    for epoch in range(1, 101):
        train(model_intrinsic, train_dataloader, optimizer, epoch)
        accuracy = test(model_intrinsic, test_dataloader)
        print("Validation Accuracy: {}".format(accuracy))
        # save information to file
        """ with open('results.txt', 'a') as f:
            f.write(f"\nEpoch: {epoch}")
            f.write(f"\nValidation Accuracy: {accuracy}")
        f.close() """
        if accuracy > best_acc:
            best_acc = accuracy
            best_epoch = epoch
            if best_acc >= 0.90:
                torch.save(
                    model_intrinsic.state_dict(), f"lenet_mnist_{intrinsic_dim}.pt"
                )
                # torch.save(model_intrinsic.state_dict(), f"model_best_h{hidden_dim}_id{intrinsic_dim}_lay{num_layers}.pt")
                """ j = None
                with open('results.json', 'r') as f:
                    j = json.load(f)
                f.close()
                with open('results.json', 'w') as f:
                    j[f"fcmodel_h{hidden_dim}_id{intrinsic_dim}_lay{num_layers}"] = {"number_parameter": num_params, "hidden_dimension": hidden_dim, "number_layers": num_layers, "intrinsic_dimension": intrinsic_dim, "epoch": epoch, "validation_accuracy": accuracy}
                    json.dump(j, f, indent=4, separators=(',', ': ')) 
                break """

    """ j = None
    with open('results_lenet.json', 'r') as f:
        j = json.load(f)
    f.close()
    with open('results_lenet.json', 'w') as f:
        j[f"lenet_model_id{intrinsic_dim}_lr{learning_rate}"] = {"number_parameter": num_params, "intrinsic_dimension": intrinsic_dim, "epoch": epoch, 
        "validation_accuracy": accuracy, "learning_rate": learning_rate, "best_epoch": best_epoch, "best_accuracy": best_acc}
        json.dump(j, f, indent=4, separators=(',', ': '))
    f.close() """

    """ j = None
    with open('results_lenet.json', 'r') as f:
        j = json.load(f)
    f.close()
    with open('results_lenet.json', 'w') as f:
        j[f"fcmodel_h{hidden_dim}_id{intrinsic_dim}_lay{num_layers}_lr{learning_rate}"] = {"number_parameter": num_params, 
        "hidden_dimension": hidden_dim, "number_layers": num_layers, "intrinsic_dimension": intrinsic_dim, "epoch": epoch, 
        "validation_accuracy": accuracy, "learning_rate": learning_rate, "best_epoch": best_epoch, "best_accuracy": best_acc}
        json.dump(j, f, indent=4, separators=(',', ': '))
    f.close() """
"""     j = None
    with open('results.json', 'r') as f:
        j = json.load(f)
    f.close()
    with open('results.json', 'w') as f:
        j[f"fcmodel_h{hidden_dim}_id{intrinsic_dim}_lay{num_layers}"] = {"number_parameter": num_params, "hidden_dimension": hidden_dim, "number_layers": num_layers, "intrinsic_dimension": intrinsic_dim, "epoch": epoch, "validation_accuracy": accuracy}
        json.dump(j, f, indent=4, separators=(',', ': ')) """

  0%|          | 0/469 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 4.00 GiB (GPU 0; 8.00 GiB total capacity; 5.27 GiB already allocated; 1.29 GiB free; 5.35 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [60]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=[1, 3.2, 5.4, 7.6, 9.8],
        y=[1, 3.2, 5.4, 7.6, 9.8],
        mode="markers",
        marker=dict(color=[1, 2, 3, 4, 5], size=30, showscale=True),
        error_y=dict(
            type="data",  # value of error bar given in data coordinates
            array=[1, 2, 3, 4, 5],
            visible=True,
        ),
        name="layer width: 50",
    )
)

fig.add_trace(
    go.Scatter(
        x=[1, 3.2, 5.4, 7.6, 9.8],
        y=[1, 3.2, 5.4, 7.6, 9.8],
        mode="markers",
        marker=dict(color=[1, 2, 3, 4, 5], size=55, showscale=True),
        error_y=dict(
            type="data",  # value of error bar given in data coordinates
            array=[1, 2, 3, 4, 5],
            visible=True,
        ),
        name="layer width: 100",
    )
)

fig.add_trace(
    go.Scatter(
        x=[1, 3.2, 5.4, 7.6, 9.8],
        y=[1, 3.2, 5.4, 7.6, 9.8],
        mode="markers",
        marker=dict(color=[1, 2, 3, 4, 5], size=70, showscale=True),
        error_y=dict(
            type="data",  # value of error bar given in data coordinates
            array=[1, 2, 3, 4, 5],
            visible=True,
        ),
        name="layer width: 200",
    )
)

fig.add_trace(
    go.Scatter(
        x=[1, 3.2, 5.4, 7.6, 9.8],
        y=[1, 3.2, 5.4, 7.6, 9.8],
        mode="markers",
        marker=dict(color=[1, 2, 3, 4, 5], size=90, showscale=True),
        error_y=dict(
            type="data",  # value of error bar given in data coordinates
            array=[1, 2, 3, 4, 5],
            visible=True,
        ),
        name="layer width: 400",
    )
)

fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

fig.show()

In [12]:
a = (
    torch.tensor(
        [
            [
                [0.9655, 0.0582, -0.4156],
                [0.9375, 0.8889, 0.8146],
                [1.3595, 0.6110, 0.8078],
            ],
            [
                [-1.6604, 0.1609, 0.1043],
                [-0.5520, 0.2414, 0.0949],
                [-0.1879, -0.5083, -0.8864],
            ],
        ]
    )
    .to_sparse()
    .requires_grad_(False)
)
print(a)

b = torch.tensor([[-1.3109], [0.8592], [-0.2531]], requires_grad=True)
print(b)

y = torch.bmm(a, b.broadcast_to((a.shape[0], b.shape[0], b.shape[-1])))
print(y)
print(y.is_sparse)

y.sum().backward()
print(b.grad)

tensor(indices=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                       [0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2],
                       [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]]),
       values=tensor([ 0.9655,  0.0582, -0.4156,  0.9375,  0.8889,  0.8146,
                       1.3595,  0.6110,  0.8078, -1.6604,  0.1609,  0.1043,
                      -0.5520,  0.2414,  0.0949, -0.1879, -0.5083, -0.8864]),
       size=(2, 3, 3), nnz=18, layout=torch.sparse_coo)
tensor([[-1.3109],
        [ 0.8592],
        [-0.2531]], requires_grad=True)
tensor([[[-1.1105],
         [-0.6714],
         [-1.4617]],

        [[ 2.2885],
         [ 0.9070],
         [ 0.0339]]], grad_fn=<BmmBackward0>)
False
tensor([[0.8622],
        [1.4521],
        [0.5196]])


In [10]:
a = torch.tensor(
    [
        [[0.9655, 0.0582, -0.4156], [0.9375, 0.8889, 0.8146], [1.3595, 0.6110, 0.8078]],
        [
            [-1.6604, 0.1609, 0.1043],
            [-0.5520, 0.2414, 0.0949],
            [-0.1879, -0.5083, -0.8864],
        ],
    ]
).requires_grad_(False)
print(a)

b = torch.tensor([[-1.3109], [0.8592], [-0.2531]], requires_grad=True)
print(b)

y = torch.matmul(a, b)
print(y)

y.sum().backward()
print(b.grad)

tensor([[[ 0.9655,  0.0582, -0.4156],
         [ 0.9375,  0.8889,  0.8146],
         [ 1.3595,  0.6110,  0.8078]],

        [[-1.6604,  0.1609,  0.1043],
         [-0.5520,  0.2414,  0.0949],
         [-0.1879, -0.5083, -0.8864]]])
tensor([[-1.3109],
        [ 0.8592],
        [-0.2531]], requires_grad=True)
tensor([[[-1.1105],
         [-0.6714],
         [-1.4617]],

        [[ 2.2885],
         [ 0.9070],
         [ 0.0339]]], grad_fn=<UnsafeViewBackward0>)
tensor([[0.8622],
        [1.4521],
        [0.5196]])
